# 修正 `air_pollution_score` 数据类型
- 2008：将字符串转换成浮点
- 2018：将整数转换成浮点

In [37]:
import pandas as pd
# 加载数据集
df_08 = pd.read_csv('data_08_duplicated.csv')
df_18 = pd.read_csv('data_18_duplicated.csv')

In [38]:
df_08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986 entries, 0 to 985
Data columns (total 13 columns):
model                   986 non-null object
displ                   986 non-null float64
cyl                     986 non-null int64
trans                   986 non-null object
drive                   986 non-null object
fuel                    986 non-null object
veh_class               986 non-null object
air_pollution_score     986 non-null object
city_mpg                986 non-null object
hwy_mpg                 986 non-null object
cmb_mpg                 986 non-null object
greenhouse_gas_score    986 non-null object
smartway                986 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 100.2+ KB


In [39]:
# 尝试使用 Pandas 的 to_numeric 或 astype 功能将
# 2008 air_pollution_score 列转换成浮点——无法转换
df_08['air_pollution_score']=df_08['air_pollution_score'].astype(float)

ValueError: could not convert string to float: '6/4'

# 解决问题
这个问题看起来不像数据类型转换那样简单。根据上述错误，第 582 行的值是 "6/4"。我们检验一下。

In [40]:
df_08.iloc[582]

model                   MERCEDES-BENZ C300
displ                                    3
cyl                                      6
trans                              Auto-L7
drive                                  2WD
fuel                           ethanol/gas
veh_class                        small car
air_pollution_score                    6/4
city_mpg                             13/18
hwy_mpg                              19/25
cmb_mpg                              15/21
greenhouse_gas_score                   7/6
smartway                                no
Name: 582, dtype: object

# 存在这个问题的不只是空气污染分数！
mpg 列和温室气体分数也存在同样的问题。这可能是它们都被保存为字符串的原因。根据我在 PDF 文档中找到的 [这个链接](http://www.fueleconomy.gov/feg/findacarhelp.shtml#airPollutionScore)，

> “如果车辆采用多种燃料提供动力，应该提供每个燃料类型的估值。”

所以，采用混合动力的所有车辆，例如上面这辆车（采用乙醇和天然气），它们的字符串具有两个值，每种燃料为一个值。这有点复杂，我将给你展示如何用 2008 数据集实现这个效果，之后你用 2018 数据集尝试一下。

使用 `df['列名'].str.contains('xx')` 先找到df中的某一列是否具有某些值

In [41]:
# 首先，获取 2008 年的所有混合动力
hb_08 = df_08[df_08['fuel'].str.contains('/')]
hb_08['fuel']

582    ethanol/gas
Name: fuel, dtype: object

好像这个数据集只有一个混合动力车，而 2018 年有很多。但是别担心，现在进行的步骤对它同样适用！

In [42]:
# 2018年的混合动力
hb_18 = df_18[df_18['fuel'].str.contains('/')]
len(hb_18)

38

提取每个混合动力行，把它们分成两个新行，一行是第一种燃料类型的值（"/" 前面的值），另一行是第二种燃料类型的值（"/" 后面的值）。现在用两个数据框把它们分开。

In [43]:
# 创建 2008 混合动力数据框的两个副本
df1 = hb_08.copy()  # 每个混合动力车第一种燃料类型的数据
df2 = hb_08.copy()  # 每个混合动力车第二种燃料类型的数据

# 每个数据应该如下所示
df1

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,ethanol/gas,small car,6/4,13/18,19/25,15/21,7/6,no


接下来，我们将使用 Pandas 的 apply 函数。参见 [这里](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) 的文档。  
* 注意：apply函数

In [44]:
# 将被 "/" 分割的列
split_columns = ['fuel', 'air_pollution_score', 'city_mpg', 'hwy_mpg', 'cmb_mpg', 'greenhouse_gas_score']

# 对每个数据框副本的每个列应用分割功能
for c in split_columns:
    df1[c] = df1[c].apply(lambda x: x.split("/")[0])
    df2[c] = df2[c].apply(lambda x: x.split("/")[1])

In [45]:
# 这个数据框包含混合动力车第一种燃料类型的信息
# 也就是 "/" 前面的值
df1

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,ethanol,small car,6,13,19,15,7,no


In [46]:
# 这个数据框包含混合动力车第二种燃料类型的信息
# 也就是 "/" 后面的值
df2

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,gas,small car,4,18,25,21,6,no


In [47]:
# 将数据框组合，添加到原始数据框中
new_rows = df1.append(df2)

# 现在每辆车的每种燃料类型都有单独的行
new_rows

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,ethanol,small car,6,13,19,15,7,no
582,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,gas,small car,4,18,25,21,6,no


In [48]:
# 丢弃原始混合动力行
df_08.drop(hb_08.index, inplace=True)

* 注意df.append方法

In [49]:
# 添加新分割的行
df_08 = df_08.append(new_rows, ignore_index=True)

In [50]:
# 检查含有 "/" 的所有原始混合动力行是否都已删除
df_08[df_08['fuel'].str.contains('/')]

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [51]:
df_08.shape

(987, 13)

# 对 2018 数据集重复相同过程

In [52]:
hb_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 52 to 793
Data columns (total 13 columns):
model                   38 non-null object
displ                   38 non-null float64
cyl                     38 non-null float64
trans                   38 non-null object
drive                   38 non-null object
fuel                    38 non-null object
veh_class               38 non-null object
air_pollution_score     38 non-null int64
city_mpg                38 non-null object
hwy_mpg                 38 non-null object
cmb_mpg                 38 non-null object
greenhouse_gas_score    38 non-null int64
smartway                38 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 4.2+ KB


In [53]:
# 创建 2018 混合动力数据框 hb_18 的两个副本
df3 = hb_18.copy()
df4 = hb_18.copy()

### 分割 `fuel`、`city_mpg`、`hwy_mpg`、`cmb_mpg` 的值
不需要分割 `air_pollution_score` 或 `greenhouse_gas_score` 的值，因为这些列在 2018 数据集中已经是整数。

In [57]:
df3.head()

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
52,BMW 330e,2.0,4.0,SemiAuto-8,2WD,Gasoline/Electricity,small car,3,28/66,34/78,30/71,10,Yes
78,BMW 530e,2.0,4.0,SemiAuto-8,2WD,Gasoline/Electricity,small car,7,27/70,31/75,29/72,10,Elite
79,BMW 530e,2.0,4.0,SemiAuto-8,4WD,Gasoline/Electricity,small car,7,27/66,31/68,28/67,10,Elite
92,BMW 740e,2.0,4.0,SemiAuto-8,4WD,Gasoline/Electricity,large car,3,25/62,29/68,27/64,9,Yes
189,CHEVROLET Impala,3.6,6.0,SemiAuto-6,2WD,Ethanol/Gas,large car,5,14/18,20/28,16/22,4,No


In [58]:
# 待分割列的列表
split_columns = ['fuel','city_mpg','hwy_mpg','cmb_mpg']

# 对每个数据框副本的每个列应用分割功能
for c in split_columns:
    df3[c] = df3[c].apply(lambda x:x.split("/")[0])
    df4[c] = df4[c].apply(lambda x:x.split("/")[1])


In [59]:
df3.head(2)

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
52,BMW 330e,2.0,4.0,SemiAuto-8,2WD,Gasoline,small car,3,28,34,30,10,Yes
78,BMW 530e,2.0,4.0,SemiAuto-8,2WD,Gasoline,small car,7,27,31,29,10,Elite


In [60]:
df4.head(2)

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
52,BMW 330e,2.0,4.0,SemiAuto-8,2WD,Electricity,small car,3,66,78,71,10,Yes
78,BMW 530e,2.0,4.0,SemiAuto-8,2WD,Electricity,small car,7,70,75,72,10,Elite


In [61]:
# 添加这两个数据框
new_rows = df3.append(df4)

# 从原始 2018 数据框中丢弃每个混合动力行
# 对 hb_18 的索引进行 Pandas 的丢弃功能，实现上述步骤
# hb_18为18年的混动列表
df_18.drop(hb_18.index, inplace=True)

# 将 new_rows 添加到 df_18
df_18 = df_18.append(new_rows)

In [62]:
# 检查是否已删除
df_18[df_18['fuel'].str.contains('/')]

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [63]:
df_18.shape

(832, 13)

### 现在可以继续进行 `air_pollution_score` 所需的更改，如下所示：
- 2008：将字符串转换成浮点
- 2018：将整数转换成浮点

In [68]:
len(df_08[df_08['air_pollution_score'].str.contains('/')])

0

In [69]:
df_08['air_pollution_score']=df_08['air_pollution_score'].astype(float)

In [70]:
df_08.head(2)

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
0,ACURA MDX,3.7,6,Auto-S5,4WD,Gasoline,SUV,7.0,15,20,17,4,no
1,ACURA RDX,2.3,4,Auto-S5,4WD,Gasoline,SUV,7.0,17,22,19,5,no


In [64]:
# 对于 2008 空气污染列，将字符串转换成浮点
aps_08=df_08[df_08['air_pollution_score'].str.contains('/')]
aps_08['air_pollution_score']

Series([], Name: air_pollution_score, dtype: object)

In [77]:
# len(df_18[df_18['air_pollution_score'].str.contains('/')])
# len(df_18[df_18['air_pollution_score'].str.contains('/')])
# df_18['air_pollution_score']

In [74]:
# 对于 2018 空气污染列，将整数转换成浮点
df_18['air_pollution_score']=df_18['air_pollution_score'].astype(float)

In [75]:
df_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 832 entries, 0 to 793
Data columns (total 13 columns):
model                   832 non-null object
displ                   832 non-null float64
cyl                     832 non-null float64
trans                   832 non-null object
drive                   832 non-null object
fuel                    832 non-null object
veh_class               832 non-null object
air_pollution_score     832 non-null float64
city_mpg                832 non-null object
hwy_mpg                 832 non-null object
cmb_mpg                 832 non-null object
greenhouse_gas_score    832 non-null int64
smartway                832 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 131.0+ KB


In [78]:
df_08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987 entries, 0 to 986
Data columns (total 13 columns):
model                   987 non-null object
displ                   987 non-null float64
cyl                     987 non-null int64
trans                   987 non-null object
drive                   987 non-null object
fuel                    987 non-null object
veh_class               987 non-null object
air_pollution_score     987 non-null float64
city_mpg                987 non-null object
hwy_mpg                 987 non-null object
cmb_mpg                 987 non-null object
greenhouse_gas_score    987 non-null object
smartway                987 non-null object
dtypes: float64(2), int64(1), object(10)
memory usage: 100.3+ KB


In [76]:
df_08.to_csv('data_08_convert_part2.csv', index=False)
df_18.to_csv('data_18_convert_part2.csv', index=False)

* 注意此输出的结果已经为正确结果！！